In [1]:
#!jupyter nbconvert --to=python MLD_Comparison_python3.ipynb

In [20]:
class MLD_comp:
    '''
    class MLD_comp(runname,resultpath,savepath,mesh,matfileMLD,first_year,last_year,
                 mapproj='pc',savefig=False, verbose=False, output=False, 
                            plotting=True, Taylor=True)
                 
    n_levels = 1: number of mesh levels used for FESOM surface mean
    
    self.MLDfesom_interp contains 2D dataset of 1x1 interpolated MLD
    self.MLDobs contains 2D dataset of 1x1 interpolated of in situ MLD climatology
    self.lon longitude
    self.lat latitude
    '''
    
    def __init__(self,resultpath,savepath,mesh,matfileMLD,first_year,last_year,
                 mapproj='rob',cmap_extension='max',cmap = 'viridis',
                 savefig=False,runname='fesom',
                 output=False,plotting=True, verbose = False, Taylor=True):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.matfileMLD=matfileMLD
        self.verbose = verbose
        self.taylor = Taylor
        self.output = output
        self.plotting = plotting
        self.Taylor = Taylor
        self.cmap = cmap
        self.cmap_extension = cmap_extension

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import scipy.io as spio
        import cartopy.feature as cfeature
        import pyfesom2 as pf
        from Py_f2recom_toolbox import plt_Taylor_norm
        from load_interp_MLD import MLDdata
        from Py_f2recom_toolbox import load_mat
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)

        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))
        
        # load Atlas MLD data -------------------------------------------------------------------------------------
        
        MLDinput = MLDdata(runname,resultpath,mesh,matfileMLD)
        MLD_sept_int = np.array(MLDinput.mld_sept_int)
        MLD_marc_int = np.array(MLDinput.mld_marc_int)
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(self.meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)      
        
        MLDfesom = pf.get_data(self.resultpath, "MLD2", years, mesh, 
                               how=None, compute=False, runid=self.runname, silent=True)
        #MLD = MLD.resample(time='MS').mean(dim='time').compute()
        MLDfesom = -MLDfesom.groupby('time.month').mean('time')
        
        MLDfesom_marc = np.array(MLDfesom[2,:])
        MLDfesom_sept = np.array(MLDfesom[8,:])
        
        #labelfesom = 'FESOM {0}-{1}'.format(self.fyear,self.lyear)
        labelfesom = 'FESOM'
        labelAtlas = 'Atlas'
        unit = 'MLD [m]'
        
        
        # apply sea mask to MLD as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        MLD_sept_int_ma = np.copy(MLD_sept_int)
        MLD_sept_int_ma[~np.isfinite(MLDfesom_sept)] = np.nan
        
        MLD_marc_int_ma = np.copy(MLD_marc_int)
        MLD_marc_int_ma[~np.isfinite(MLDfesom_marc)] = np.nan
        
        
        # check CHl.a data -------------------------------------------------------------------------------------
        if(self.verbose):
            print('\nWinter MLD\nAtlas nmin = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f}'.format(
                    np.nanmin(MLD_marc_int_ma),np.nanmax(MLD_marc_int_ma),
                    np.nanmin(MLDfesom_marc),np.nanmax(MLDfesom_marc)))
            print('\nSummer MLD\nAtlas nmin = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f}'.format(
                    np.nanmin(MLD_sept_int_ma),np.nanmax(MLD_sept_int_ma),
                    np.nanmin(MLDfesom_sept),np.nanmax(MLDfesom_sept)))
        
        if plotting:

            levels = np.array([0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                               1,2,3,4,5,6,7,8])*100
            ticks = [0.05,0.1,0.3,0.5,0.7,1,3,5,6,7,8]*100
            ticks_label = ['0.05','0.1','0.3','0.5','0.7','1','3','7'] # +7 ?
            # plot each dataset -------------------------------------------------------------------------------------        
            #levels = np.arange(0,620,20)
            levels_diff = np.arange(-400,420,20)

            fig = plt.figure(figsize=(12,5), constrained_layout=True)
            axes = fig.subplot_mosaic(
                    """
                    ABC
                    DEF
                    """,
                    gridspec_kw={'hspace': 0.01, 'wspace': 0.01}, 
                    subplot_kw=dict(projection=self.mapproj))             
            fig.get_layout_engine().set(w_pad=0.2)
            
            # FESOM 
            m1 = axes['A']
            f1 = pf.subplot(mesh, fig, m1, [MLDfesom_marc],
                                levels = levels,
                                units=unit, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelfesom,
                                box= box, ptype="cflog",
                               )

            # Atlas 
            m2 = axes['B']
            f2 = pf.subplot(mesh, fig, m2, [MLD_marc_int_ma],
                                levels = levels,
                                units=unit, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelAtlas,
                                box= box, ptype="cflog",
                               )

            # FESOM - Atlas
            m3 = axes['C']
            f3 = pf.subplot(mesh, fig, m3, [MLDfesom_marc - MLD_marc_int_ma],
                                levels = levels_diff,
                                units=unit, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                titles='FESOM - Atlas',
                                box= box, ptype="cflog",
                               )
            
            


            # FESOM
            m4 = axes['D']
            f4 = pf.subplot(mesh, fig, m4, [MLDfesom_sept],
                                levels = levels,
                                units=unit, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                box= box, ptype="cflog",
                               )

            # Atlas 

            m5 = axes['E']
            f5 = pf.subplot(mesh, fig, m5, [MLD_sept_int_ma],
                                levels = levels,
                                units=unit, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                box= box, ptype="cflog",
                               )
            
            # FESOM - Atlas 
            m6 = axes['F']
            f6 = pf.subplot(mesh, fig, m6, [MLDfesom_sept - MLD_sept_int_ma],
                                levels = levels_diff,
                                units=unit, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                box= box, ptype="cflog",
                               )
            
            fig.subplots_adjust(bottom=0.2)
            
            cbar1_ax = fig.add_axes([0.10, 0.0001, 0.52, 0.02])
            cbar1 = fig.colorbar(f5,
                            cax = cbar1_ax, 
                            orientation = 'horizontal',
                            #ticks = ticks,
                            fraction=0.1, pad=0.1) 
            cbar1.set_label(unit, fontsize=16)
            cbar1.ax.tick_params(labelsize=16)
            

            cbar2_ax = fig.add_axes([0.75, 0.0001, 0.21, 0.02])
            cbar2 = fig.colorbar(f6,
                            cax = cbar2_ax, 
                            orientation = 'horizontal',
                            #location ='bottom',
                            ticks = [-400,-200,0,200,400],
                                )
            cbar2.ax.tick_params(labelsize=16)
            cbar2.set_label(unit, fontsize=16)
            
            
            m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                        size=30, weight='bold')
            m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                        size=30, weight='bold')
            m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                        size=30, weight='bold')
            m4.text(-0.12, 1.05, 'D', transform=m4.transAxes,
                        size=30, weight='bold')
            m5.text(-0.12, 1.05, 'E', transform=m5.transAxes,
                        size=30, weight='bold')
            m6.text(-0.12, 1.05, 'F', transform=m6.transAxes,
                        size=30, weight='bold')
            # add one colorbar for difference plot below figure

            m1.text(-0.18, .3, 'March', transform=m1.transAxes,
                        size=18, rotation=90)
            m4.text(-0.18, .05, 'September', transform=m4.transAxes,
                        size=18, rotation=90)
            
            
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'MLD'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'MLD'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)  
            

#        if(self.Taylor):
#             # statistics  -------------------------------------------------------------------------------------            
#             # preparation of datasets
#             if np.isnan(np.min(OCNPP_ma_log10)): print('WARNING: OCNPP field contains NaNs')
#             if np.isnan(np.min(NPPt_log10)): print('WARNING: FESOM field contains NaNs')

#             # get statistics only from valid OCCCI gridpoints 
#             ind_stat = np.where(np.isfinite(OCNPP_ma_log10))

#             title = 'MLD'
#             print('\nStatistics for '+title)
#             plt_Taylor_norm(OCNPP_ma_log10[ind_stat],NPPt_log10[ind_stat],
#                                         mask=True,title=title)

#             # fig export  -------------------------------------------------------------------------------------
#             if(self.savefig==True):                
#                 plt.savefig(self.savepath+self.runname+'_'+'OCNPP_Taylor'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
#                             dpi = 300, bbox_inches='tight')
#             plt.show(block=False) 
        
#        # store interpolated datasets ----------------------------------------------------------------------------------
#        if output:
#             self.NPPd_interp = NPPd_interp
#             self.NPPn_interp = NPPn_interp
#             self.NPPt_interp = NPPt_interp
#             self.unit = unitfesom
#             self.NPPt_OC = OCNPP_ma